In [1]:
#le but de cette partie est de predire les chiffres sur une nouvelle image 
#qui contient une suite de degits manuscrits à partir du modéle SVM sauvegardé
#predire la classe de chaque digits

In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import joblib
from skimage.feature import hog 
import numpy as np

In [20]:
#chargement de modéle
model = joblib.load("svm_gigits.pkl")

In [25]:
image = cv2.imread('test.jpg')
cv2.imshow('original image',image)
cv2.waitKey(0)
#cv2.destroyAllwindows()

-1

In [6]:
#pour faire la prediction ,on dois separer et decouper chaque degit et predire chaque degits aparts

In [7]:
#convertir l'image en niveau de gris
im_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("image en niveaux de gris",im_gray)
cv2.waitKey(0)

-1

In [8]:
pip install opencv-contrib-python


  Using cached opencv_contrib_python-4.4.0.46-cp38-cp38-win_amd64.whl (40.2 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Accès refusé: 'C:\\Users\\boukh\\anaconda3\\Lib\\site-packages\\cv2\\cv2.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [9]:
#application de filtre gaussian pour eliminer le bruit 
im_gaussian = cv2.GaussianBlur(im_gray,(5,5), 0)
cv2.imshow("image filtrée ",im_gray)
cv2.waitKey(0)

-1

In [10]:
#binariser l'mage pour trouver les contours
ret, im_th = cv2.threshold(im_gaussian,90,255,cv2.THRESH_BINARY_INV)#si >90 O sinon 1
cv2.imshow('Image Binaire',im_th)
cv2.imshow("image binaire",im_th)
cv2.waitKey(0)
#cv2.destroyALLwindows()

-1

In [11]:
#trouver les contours dans une image
ctrs, hier =cv2.findContours(im_th.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)#au lieu d'avoir tous le spoints de contour,on vas avoir que 
#les points interessants qui vont me delimiter le contour, dans le cas d'un rectangle, il faut que 2 points pour dessiner le rectangle
#dont l ebut de ne pas aborder le memoire


In [12]:
#dessiner un rectangle pour chaque contour
rects=[cv2.boundingRect(ctr) for ctr in ctrs]#boundingRect il nous genere x,y,w(la largeur),h(la hauteur) pour dessiner le 2eme point 
print(rects)

[(248, 268, 41, 52), (148, 266, 50, 57), (63, 262, 58, 57), (425, 261, 6, 53), (324, 260, 42, 63), (559, 257, 48, 64), (645, 256, 31, 58), (474, 256, 37, 68)]


In [31]:
for rect in rects:
    #dessiner les rectangles
    cv2.rectangle(image,(rect[0],rect[1]), (rect[0] + rect[2],rect[1]+rect[3]),(0,255,0),3)#point depart,point d'arrive,pallete de couleur,la largeur de contour
    #mettre les contour en dehors de la degit
    leng = int(rect[3]*1.6)
    pt1 = int(rect[1] + rect[3] //2 - leng // 2)
    pt2 = int(rect[0] + rect[2] //2 - leng // 2)
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng ]
    #resize the image
    roi = cv2.resize(roi, (28,28), interpolation=cv2.INTER_AREA)#on doit faire resize pour la nouvelle region,cette technique de l'interpolation,il fait un resize mais en se basant sur la surface
    roi = cv2.dilate(roi,(3, 3))
    #calculate the hog features
    roi_hog_fd = hog(roi, orientations=9 , pixels_per_cell=(14,14), cells_per_block =(1,1))
    nbr = model.predict(np.array([roi_hog_fd], 'float64'))
    cv2.putText(image,str(int(nbr)), (rect[0],rect[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0,255,255), 3)
    

In [32]:
cv2.imshow("le resultat de SVM, image)
cv2.waitKey(0)
cv2.destroyAllWindows()